In [9]:
import json

In [10]:
obj = [[1,2,3],123,123.123,'abc',{'key1':(1,2,3),'key2':(4,5,6)}]
encodedjson = json.dumps(obj)

In [15]:
print ('the original list:\n',obj )
print ('length of obj is:',len(repr(obj)))
print ('repr(obj),replace whiteblank with *:\n', repr(obj).replace(' ','*') )
print ('json encoded,replace whiteblank with *:\n',encodedjson.replace(' ','*'))
##json.dumps  :  python data-> javascript



##json.load : javascript -> python data

the original list:
 [[1, 2, 3], 123, 123.123, 'abc', {'key1': (1, 2, 3), 'key2': (4, 5, 6)}]
length of obj is: 72
repr(obj),replace whiteblank with *:
 [[1,*2,*3],*123,*123.123,*'abc',*{'key1':*(1,*2,*3),*'key2':*(4,*5,*6)}]
json encoded,replace whiteblank with *:
 [[1,*2,*3],*123,*123.123,*"abc",*{"key1":*[1,*2,*3],*"key2":*[4,*5,*6]}]


In [16]:
##json.loads :解析
##需要注意的是，json字符串中的字典类型的key必须要用双引号“”json.loads()才能正常解析


decodejson = json.loads(encodedjson) 
print ('the type of decodeed obj from json:', type(decodejson)) 
print ('the obj is:\n',decodejson) 
print ('length of decoded obj is:',len(repr(decodejson)))

the type of decodeed obj from json: <class 'list'>
the obj is:
 [[1, 2, 3], 123, 123.123, 'abc', {'key1': [1, 2, 3], 'key2': [4, 5, 6]}]
length of decoded obj is: 72


In [18]:
data1 = {'b':789,'c':456,'a':123} 
data2 = {'a':123,'b':789,'c':456} 
d1 = json.dumps(data1,sort_keys=True) 
d2 = json.dumps(data2) 
d3 = json.dumps(data2,sort_keys=True) 
print ('sorted data1(d1):',d1) 
print ('unsorted data2(d2):',d2) 
print ('sorted data2(d3):',d3 )
print ('d1==d2?:',d1==d2 )
print ('d1==d3?:',d1==d3)

sorted data1(d1): {"a": 123, "b": 789, "c": 456}
unsorted data2(d2): {"b": 789, "a": 123, "c": 456}
sorted data2(d3): {"a": 123, "b": 789, "c": 456}
d1==d2?: False
d1==d3?: True


indent参数是缩进的意思，它可以使数据的存储格式更优雅、可读性更强，这是通过增加一些冗余的空格进行填充的。但是在解码（json.loads()）时，空白填充会被删除。

In [22]:
data = {'b':789,'c':456,'a':123} 
d1 = json.dumps(data,sort_keys=True,indent=4) 
print ('data len is:',len(repr(data)) )
print ('4 indented data:\n',d1 )
d2 = json.loads(d1) 
print ('decoded DATA:', repr(d2) )
print ('len of decoded DATA:',len(repr(d2)))

data len is: 30
4 indented data:
 {
    "a": 123,
    "b": 789,
    "c": 456
}
decoded DATA: {'b': 789, 'a': 123, 'c': 456}
len of decoded DATA: 30


json主要是作为一种数据通信的格式存在的，无用的空格会浪费通信带宽，适当时候也要对数据进行压缩。separator参数可以起到这样的作用，该参数传递是一个元组，包含分割对象的字符串，其实质就是将Python默认的（‘, ',': '）分隔符替换成（',',':'）

In [23]:
data = {'b':789,'c':456,'a':123} 
print ('DATA:', repr(data)) 
print ('repr(data)       :', len(repr(data)) )
print ('dumps(data)      :', len(json.dumps(data)) )
print ('dumps(data, indent=2) :', len(json.dumps(data, indent=4))) 
print ('dumps(data, separators):', len(json.dumps(data, separators=(',',':'))))

DATA: {'b': 789, 'a': 123, 'c': 456}
repr(data)       : 30
dumps(data)      : 30
dumps(data, indent=2) : 44
dumps(data, separators): 25


另一个比较有用的dumps参数是skipkeys，默认为False。 dumps方法存储dict对象时key必须是str类型，其他类型会导致TypeError异常产生，如果将skipkeys设为True则会优雅的滤除非法keys。

In [24]:
data = {'b':789,'c':456,(1,2):123} 
print('original data:',repr(data) )
print ('json encoded',json.dumps(data,skipkeys=True))

original data: {(1, 2): 123, 'b': 789, 'c': 456}
json encoded {"b": 789, "c": 456}


json模块不仅可以处理普通的python内置类型，也可以处理我们自定义的数据类型，而往往处理自定义的对象是很常用的。

如果直接通过json.dumps方法对Person的实例进行处理的话，会报错，因为json无法支持这样的自动转化。通过上面所提到的json和 python的类型转化对照表，可以发现，object类型是和dict相关联的，所以我们需要把我们自定义的类型转化为dict，然后再进行处理。这里，有两种方法可以使用。
方法一：自己写转化函数
自定义object类型和dict类型进行转化：encode-定义函数 object2dict()将对象模块名、类名以及__dict__存储在一个字典并返回;decode-定义dict2object()解析出模块名、类名、参数，创建新的对象并返回。在json.dumps()中通过default参数指定转化过程中调用的函数；json.loads()则通过 object_hook指定转化函数。
方法二：继承JSONEncoder和JSONDecoder类，覆写相关方法
JSONEncoder类负责编码，主要是通过其default函数进行转化，我们可以重载该方法。对于JSONDecoder，亦然。

In [26]:
#handling private data type 
#define class 
class Person(object): 
  def __init__(self,name,age): 
    self.name = name 
    self.age = age 
  def __repr__(self): 
    return 'Person Object name : %s , age : %d' % (self.name,self.age) 
    
    
#define transfer functions 
def object2dict(obj): 
  #convert object to a dict 
  d = {'__class__':obj.__class__.__name__, '__module__':obj.__module__} 
  d.update(obj.__dict__) 
  return d 
   
def dict2object(d): 
  #convert dict to object 
  if'__class__' in d: 
    class_name = d.pop('__class__') 
    module_name = d.pop('__module__') 
    module = __import__(module_name) 
    print ('the module is:', module) 
    class_ = getattr(module,class_name) 
    args = dict((key.encode('ascii'), value) for key, value in d.items()) #get args 
    print ('the atrribute:', repr(args)) 
    inst = class_(**args) #create new instance 
  else: 
    inst = d 
  return inst 
#recreate the default method 
class LocalEncoder(json.JSONEncoder): 
  def default(self,obj): 
    #convert object to a dict 
    d = {'__class__':obj.__class__.__name__, '__module__':obj.__module__} 
    d.update(obj.__dict__) 
    return d 
   
class LocalDecoder(json.JSONDecoder): 
  def __init__(self): 
    json.JSONDecoder.__init__(self,object_hook = self.dict2object) 
  def dict2object(self, d): 
    #convert dict to object 
    if'__class__' in d: 
      class_name = d.pop('__class__') 
      module_name = d.pop('__module__') 
      module = __import__(module_name) 
      class_ = getattr(module,class_name) 
      args = dict((key.encode('ascii'), value) for key, value in d.items()) #get args 
      inst = class_(**args) #create new instance 
    else: 
      inst = d 
    return inst 
#test function 
if __name__ == '__main__': 
  p = Person('Aidan',22) 
  print (p) 
  #json.dumps(p)#error will be throwed 
  d = object2dict(p) 
  print ('method-json encode:', d) 
   
  o = dict2object(d) 
  print ('the decoded obj type: %s, obj:%s' % (type(o),repr(o))) 
   
  dump = json.dumps(p,default=object2dict) 
  print ('dumps(default = object2dict):',dump )
  load = json.loads(dump,object_hook = dict2object) 
  print ('loads(object_hook = dict2object):',load )
  d = LocalEncoder().encode(p) 
  o = LocalDecoder().decode(d) 
   
  print ('recereated encode method: ',d )
  print ('recereated decode method: ',type(o),o)

Person Object name : Aidan , age : 22
method-json encode: {'name': 'Aidan', 'age': 22, '__module__': '__main__', '__class__': 'Person'}
the module is: <module '__main__'>
the atrribute: {b'name': 'Aidan', b'age': 22}


TypeError: __init__() keywords must be strings